In [26]:
from RTdata import get_data
from indicators import add_indicators
from Trade import go_long, go_short
import time

In [27]:
data_df = get_data('crypto_bar')

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
Initial connection to PostgreSQL DataBase
0


In [28]:
data_df.head(2)

,symbol,time,type,close,high,low,open,trade,volume,vwap
0,BTC/USD,2022-11-13 00:14:00-05:00,b,16829.4,16829.66,16825.66,16825.96,6,0.024979,16827.1982665439
1,BTC/USD,2022-11-13 00:13:00-05:00,b,16829.7,16841.01,16829.7,16833.35,50,7.602778,16838.4521991172


In [29]:
# Set 'time' index
data_index_df = data_df.set_index('time')

#Transfer datatype
data_index_df[['symbol','type']]= data_index_df[['symbol','type']].astype('str')
data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']] = data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']].astype('float')
data_index_df.dtypes

symbol     object
type       object
close     float64
high      float64
low       float64
open      float64
trade     float64
volume    float64
vwap      float64
dtype: object

In [30]:
# Add indicators.
data_indicators_df = add_indicators(data_index_df)
data_indicators_df


,symbol,type,close,high,low,open,trade,volume,vwap,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,K_9_3,D_9_3,J_9_3
time,,,,,,,,,,,,,,,,,
2022-11-13 00:14:00-05:00,BTC/USD,b,16829.40,16829.66,16825.66,16825.96,6.0,0.024979,16827.198267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:13:00-05:00,BTC/USD,b,16829.70,16841.01,16829.70,16833.35,50.0,7.602778,16838.452199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:12:00-05:00,BTC/USD,b,16829.67,16833.43,16826.90,16826.90,12.0,0.277999,16830.213876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:11:00-05:00,BTC/USD,b,16825.55,16828.00,16823.18,16823.18,13.0,0.338730,16826.233232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:10:00-05:00,BTC/USD,b,16826.64,16827.76,16823.28,16823.28,22.0,0.142402,16824.252988,16824.692665,16828.192,16831.691335,0.041589,0.278244,NaN,NaN,NaN
2022-11-13 00:09:00-05:00,BTC/USD,b,16822.23,16825.20,16822.23,16825.20,12.0,0.252991,16823.097361,16821.164355,16826.758,16832.351645,0.066485,0.095255,NaN,NaN,NaN
2022-11-13 00:08:00-05:00,BTC/USD,b,16825.20,16831.99,16825.20,16828.44,16.0,0.258182,16830.622208,16821.055244,16825.858,16830.660756,0.057088,0.431498,NaN,NaN,NaN
2022-11-13 00:07:00-05:00,BTC/USD,b,16827.09,16831.95,16823.38,16823.95,55.0,0.732512,16829.305801,16821.937499,16825.342,16828.746501,0.040469,0.756719,NaN,NaN,NaN
2022-11-13 00:06:00-05:00,BTC/USD,b,16826.90,16826.90,16818.69,16823.34,12.0,0.148001,16822.608314,16821.977952,16825.612,16829.246048,0.043197,0.677213,NaN,NaN,NaN


In [17]:
data_indicators_df.dtypes

symbol        object
type          object
close        float64
high         float64
low          float64
open         float64
trade        float64
volume       float64
vwap         float64
BBL_5_2.0    float64
BBM_5_2.0    float64
BBU_5_2.0    float64
BBB_5_2.0    float64
BBP_5_2.0    float64
dtype: object

In [8]:
# Create data acquire function:
def clean_data():
    data_df = get_data('crypto_bar')
    # Set 'time' index
    data_index_df = data_df.set_index('time')

    #Transfer datatype
    data_index_df[['symbol','type']]= data_index_df[['symbol','type']].astype('str')
    data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']] = data_index_df[['close', 'high', 'low', 'open','trade','volume','vwap']].astype('float')
    # Add indicators.
    data_indicators_df = add_indicators(data_index_df)
    return data_indicators_df

In [9]:
data_indicators_df = clean_data()
data_indicators_df[data_indicators_df['BBL_5_2.0'] >= data_indicators_df['close']]
data_indicators_df.head(25)

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
Initial connection to PostgreSQL DataBase
0


,symbol,type,close,high,low,open,trade,volume,vwap,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
time,,,,,,,,,,,,,,
2022-11-13 00:06:00-05:00,BTC/USD,b,16826.90,16826.90,16818.69,16823.34,12.0,0.148001,16822.608314,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:05:00-05:00,BTC/USD,b,16823.34,16831.82,16820.02,16830.33,18.0,0.607701,16823.773360,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:04:00-05:00,BTC/USD,b,16831.18,16831.18,16824.47,16827.38,10.0,0.349063,16826.117974,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:03:00-05:00,BTC/USD,b,16831.13,16834.18,16825.92,16834.18,26.0,0.329199,16830.686506,NaN,NaN,NaN,NaN,NaN
2022-11-13 00:02:00-05:00,BTC/USD,b,16830.48,16833.23,16825.12,16827.49,48.0,1.057035,16826.990184,16822.464396,16828.606,16834.747604,0.072990,0.652566
2022-11-13 00:01:00-05:00,BTC/USD,b,16826.72,16833.80,16819.71,16827.08,166.0,3.869651,16827.961378,16822.386849,16828.570,16834.753151,0.073484,0.350400
2022-11-13 00:00:00-05:00,BTC/USD,b,16832.62,16834.53,16827.08,16833.36,25.0,2.226050,16829.615313,16826.464675,16830.426,16834.387325,0.047073,0.776928


In [ ]:
'''
1.D％值在70以上时，市场呈现超买现象。D％值在30以下时市场则呈现超卖现象。 
2.当Ｋ％线发生倾斜度趋于平缓时，是为警告讯号。 
3.当K％值大于D％值，显示目前是向上涨升的趋势，因此在图形上K％向上突破D％线时，即为买进的讯号。 
4.当D％值大于Ｋ％值，显示目前的趋势是向下跌落，因此在图形上K％向下跌破D％线，即为卖出讯号。 
5.K％线与D％线的交叉，须在80以下，20以下讯号才较为正确。K％与D％线在特性上与强弱指标一样，当Ｋ％值与D％值在70以上，已显示超买的现象，30以下出现超卖的现象。唯强弱指标不能明显的显示买卖时机，而KD线则可以达到此目的。 
6.背离讯号产生时，亦为非常正确的买进、卖出时机。 
7.发行量太小，交易量太小的股票不适用KD指标；加权指数以及热门大型股准确性极高。 
8.KD值在50左右交叉且为盘局时，此指标不能当成明显的买卖讯号。
9.J％>100时为超买，J％<10时为超卖，反应较K％及D％为快。KD指标是计算ｎ日内买卖超情况，再加上平均线观念而求出的可设定计算期（即ｎ值），P1、P2、P3值。一般P1，P2都设成3，若P3设成0，表示不画J线。 
'''

In [4]:
# Set Condition to excute trade.
def trade_condition_check(qty):
    while True:
        # get clean data.
        data_indicators_df = clean_data()
        # Go long condition 1 BOLL: close <= BBL
        if data_indicators_df['close'][0] <= data_indicators_df['BBL_5_2.0'][4]:
            # set current price as low price
            low_price_point = data_indicators_df['low']
            print(f'Low price point: {low_price_point}')
            if data_indicators_df['D_9_3'][15] < 20:
                if data_indicators_df['K_9_3'][13] > data_indicators_df['D_9_3'][15]:
                    if data_indicators_df['J_9_3'][15] < 10: 
                        symbol = data_indicators_df['symbol'][0]
                        limit_price = data_indicators_df['low'][0]
                        take_profit_price = limit_price * 1.01
                        stop_loss_price = limit_price * 0.99
                        go_long(symbol,qty,limit_price,take_profit_price,stop_loss_price)
                        print('Trade placed')
                        break
        # Go short condition 1 BOLL: close >= BBL
        elif data_indicators_df['close'][0] >= data_indicators_df['BBU_5_2.0'][4]:
            high_price_point = data_indicators_df['high']
            print(f'High price point: {high_price_point}')
            # Go Short condition 2 - KDJ 
            if data_indicators_df['D_9_3'][15] > 80:
                if data_indicators_df['K_9_3'][13] < data_indicators_df['D_9_3'][15]:
                    if data_indicators_df['J_9_3'][15] > 100:
                        # prepare trade detail
                        symbol = data_indicators_df['symbol'][0]
                        limit_price = data_indicators_df['high'][0]
                        take_profit_price = limit_price * 0.99
                        stop_loss_price = limit_price * 1.01
                        # submit trade
                        go_short(symbol,qty,limit_price,take_profit_price,stop_loss_price)
                        print('Trade placed')
                        break
        else:
            print(f"***Monitoring the trade condition***\n"
                  f"Ticker: {data_indicators_df['symbol'][0]}\n"
                  f"Close: {data_indicators_df['close'][0]}\n"
                  f"High: {data_indicators_df['high'][0]}\n"
                  f"Low: {data_indicators_df['low'][0]}\n"
                  f"BBL: {data_indicators_df['BBL_5_2.0'][4]}\n"
                  f"BBU: {data_indicators_df['BBU_5_2.0'][4]}\n"
                  f"K: {data_indicators_df['K_9_3'][13]}\n"
                  f"D: {data_indicators_df['D_9_3'][15]}\n"
                  f"J: {data_indicators_df['K_9_3'][13]}"
                 )

            
        time.sleep(60)

In [7]:
trade_condition_check(qty = 1)

DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
Initial connection to PostgreSQL DataBase
0
***Monitoring the trade condition***
Ticker: BTC/USD
Close: 16895.85
High: 16896.12
Low: 16886.39
BBL: 16891.327742962545
BBU: 16896.292257037458
K: 44.322159503014326
D: 48.17294089218857
J: 44.322159503014326
DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
Initial connection to PostgreSQL DataBase
0
***Monitoring the trade condition***
Ticker: BTC/USD
Close: 16883.33
High: 16896.69
Low: 16883.33
BBL: 16882.971849385136
BBU: 16900.50015061486
K: 34.258924094473734
D: 47.51435783321074
J: 34.258924094473734
DataBase Connected!
Version: PostgreSQL 14.5 (Homebrew) on aarch64-apple-darwin22.1.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.202), 64-bit
Initial connection to P

UndefinedTable: relation "_2022_11_13_crypto_bar" does not exist
LINE 1: SELECT * FROM _2022_11_13_crypto_bar ORDER BY time DESC LIMI...
                      ^
